<a href="https://colab.research.google.com/github/HunterWelsch/BigData-AWS-RDS/blob/master/Challange_16_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [0]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-02-21 04:46:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.69MB/s    in 0.2s    

2020-02-21 04:46:00 (3.69 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
!pip install pyspark

     |████████████████████████████████| 217.8MB 59kB/s 
     |████████████████████████████████| 204kB 51.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=5c08c82d97e0ec4d4a5c2d221752ffc2888b52d0267a41229ddee0411d3bc031
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
e_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True)
e_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [0]:
vine_e = e_data_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_e.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            

In [0]:
vine_e.count()

3093869

In [0]:
from pyspark.sql.types import IntegerType, DateType, BooleanType
e_bool_df = e_data_df.withColumn("vine", e_data_df["vine"].cast(BooleanType()))

In [0]:
vine_e = e_bool_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_e.show()

+--------------+-----------+-------------+-----------+-----+
|     review_id|star_rating|helpful_votes|total_votes| vine|
+--------------+-----------+-------------+-----------+-----+
|R2MTG1GCZLR2DK|          5|            0|          0|false|
|R2HBOEM8LE9928|          5|            0|          0|false|
|R1P4RW1R9FDPEE|          5|            1|          1|false|
|R1EBPM82ENI67M|          1|            0|          0|false|
|R372S58V6D11AT|          5|            1|          1|false|
|R1A4514XOYI1PD|          5|            1|          1|false|
|R20D9EHB7N20V6|          5|            0|          0|false|
|R1WUTD8MVSROJU|          5|            0|          0|false|
|R1QCYLT25812DM|          4|            0|          0|false|
| R904DQPBCEM7A|          4|            0|          0|false|
|R1DGA6UQIVLKZ7|          5|            0|          0|false|
| RLQT3V8SMNIBH|          5|            0|          0|false|
|R3T9GZS2TMXZGM|          1|            0|          0|false|
|R24HVAEYP5PLDN|        

In [0]:
vine_e_true = vine_e.filter(vine_e['vine'] == True)
vine_e_true.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1T7X1RPWYTRBU|          5|            2|          4|true|
| RJFGZBIFXCC65|          5|            0|          0|true|
| RSZ6RLTTGWUN6|          5|            0|          1|true|
|R2N4GVP3Q5PF9Y|          3|            2|          3|true|
| RCAFL1X52MFZM|          5|            1|          2|true|
|R17D8E4KO0FS83|          5|            1|          1|true|
|R184FOUNZZ7KO8|          5|           15|         20|true|
|R2SP52AJMIERZ5|          3|            0|          0|true|
|R15VB3EZ2C0W3J|          4|            0|          0|true|
|R3CMY924EQEGIU|          4|            0|          0|true|
|R2T1AAO75DGP6C|          4|            0|          0|true|
|R2AYZV2RYCKIDO|          4|            1|          1|true|
|R3BEI48BY6XIRV|          3|            0|          0|true|
|R2PLGJM0MUI41A|          5|            

In [0]:
vine_e_false = vine_e.filter(vine_e['vine'] == False)
vine_e_false.show()

+--------------+-----------+-------------+-----------+-----+
|     review_id|star_rating|helpful_votes|total_votes| vine|
+--------------+-----------+-------------+-----------+-----+
|R2MTG1GCZLR2DK|          5|            0|          0|false|
|R2HBOEM8LE9928|          5|            0|          0|false|
|R1P4RW1R9FDPEE|          5|            1|          1|false|
|R1EBPM82ENI67M|          1|            0|          0|false|
|R372S58V6D11AT|          5|            1|          1|false|
|R1A4514XOYI1PD|          5|            1|          1|false|
|R20D9EHB7N20V6|          5|            0|          0|false|
|R1WUTD8MVSROJU|          5|            0|          0|false|
|R1QCYLT25812DM|          4|            0|          0|false|
| R904DQPBCEM7A|          4|            0|          0|false|
|R1DGA6UQIVLKZ7|          5|            0|          0|false|
| RLQT3V8SMNIBH|          5|            0|          0|false|
|R3T9GZS2TMXZGM|          1|            0|          0|false|
|R24HVAEYP5PLDN|        

In [0]:
vine_e.filter(vine_e['vine'] == True).count()

18512

In [0]:
vine_e.filter(vine_e['vine'] == False).count()

3075349

In [0]:
vine_e.filter(vine_e['star_rating'] == "5").show()

In [0]:
vine_e_true.filter(vine_e_true['star_rating'] == "5").count()

8044

In [0]:
vine_e_false.filter(vine_e_false['star_rating'] == "5").count()

1773112

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
vg_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
vg_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [0]:
vg_bool_df = vg_data_df.withColumn("vine", vg_data_df["vine"].cast(BooleanType()))

In [0]:
vine_vg = vg_bool_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_vg.show()

+--------------+-----------+-------------+-----------+-----+
|     review_id|star_rating|helpful_votes|total_votes| vine|
+--------------+-----------+-------------+-----------+-----+
| RSH1OZ87OYK92|          2|            2|          3|false|
|R1WFOQ3N9BO65I|          5|            0|          0|false|
| R3YOOS71KM5M9|          5|            0|          0|false|
|R3R14UATT3OUFU|          5|            0|          0|false|
| RV2W9SGDNQA2C|          5|            0|          0|false|
|R3CFKLIZ0I2KOB|          5|            0|          0|false|
|R1LRYU1V0T3O38|          4|            0|          0|false|
| R44QKV6FE5CJ2|          5|            0|          0|false|
|R2TX1KLPXXXNYS|          5|            0|          0|false|
|R1JEEW4C6R89BA|          5|            0|          0|false|
|R3B3UHK1FO0ERS|          1|            0|          0|false|
|R2GVSDHW513SS1|          5|            0|          0|false|
|R1R1NT516PYT73|          5|            0|          0|false|
|R3K624QDQKENN9|        

In [0]:
vine_vg.filter(vine_vg['vine'] == True).show()

+---------+-----------+-------------+-----------+----+
|review_id|star_rating|helpful_votes|total_votes|vine|
+---------+-----------+-------------+-----------+----+
+---------+-----------+-------------+-----------+----+



#**Are Vine Reviews Bias?**

### Two amazon product review datasets were analyzed for positive review bias from vine reviews. One of the datasets consisted of electronic product reviews, while the second dataset consisted of video game reviews. The video game dataset did not have any vine reviews in it, leaving no bias of vine reviews by defualt. The electronic dataset had a total of 18,512 vine reviews, accounting for 0.6% of the reviews in the dataset. The other 94% of the data is regualr reviews. Within the small number of vine reviews 43% of them gave the product a five star review, compared to the regular reviewers who gave out five stars 58% of the time. 
### It has been established that vine reviews are rare, but do not show positive review bias. Vine reviewers are less likely to rate a product with five stars then thier regualr counter-parts. 
---

